# <center><p style ="color: smoke; font-weight: 500">Digit Recognizer competition</p></center>

<center>>>> Nguyễn Ngọc Trí Vĩ <<<

Link competitions: https://www.kaggle.com/competitions/digit-recognizer

### Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras.utils import plot_model
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import mnist

### Import and preprocess data

In [2]:
train_df = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")

Y_train = train_df["label"]
X_train = train_df.drop(labels=["label"], axis=1)

Import and combine Mnist data for training

In [3]:
(x, y), (xt, yt) = mnist.load_data()

train = np.concatenate([x, xt], axis=0)
y = np.concatenate([y, yt], axis=0)

X_train1 = train.reshape(-1, 28 * 28)

X_train = np.concatenate([X_train, X_train1], axis = 0)
Y_train = np.concatenate([Y_train, y], axis = 0)

#### Preprocessing train and test set

-   Normalization
-   One-hot encoding
-   Spliting data to train - validate - test set

In [4]:
X_train = X_train/255.0
X_test = X_test/255.0

X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.values.reshape(-1, 28, 28, 1)

Y_train = to_categorical(Y_train, num_classes=10)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=2)

X_train.shape, X_val.shape, Y_train.shape, Y_val.shape


((100800, 28, 28, 1), (11200, 28, 28, 1), (100800, 10), (11200, 10))

### Build, train model and predict

#### Convolutional Neural Network (CNN)

Build

In [5]:
model = Sequential(
    [
        Conv2D(filters=64, kernel_size=(5, 5), padding='Same', activation='relu', input_shape=(28, 28, 1)),
        BatchNormalization(),
        Conv2D(filters=64, kernel_size=(5, 5), padding='Same', activation='relu'),
        BatchNormalization(),
        MaxPool2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'),
        BatchNormalization(),

        Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'),
        BatchNormalization(),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Dropout(0.25),

        Conv2D(filters=64, kernel_size=(3, 3), padding='Same',  activation='relu'),
        BatchNormalization(),
        Dropout(0.25),

        Flatten(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.25),

        Dense(10, activation ='softmax'),
    ]
)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
batch_size = 128

datagen = ImageDataGenerator(
        rotation_range=10,
        zoom_range=0.10,
        width_shift_range=0.1,
        height_shift_range=0.1)

train_gen = datagen.flow(X_train, Y_train, batch_size=batch_size)

In [ ]:
epochs = 50

history = model.fit(train_gen, epochs=epochs, validation_data = (X_val, Y_val), verbose=1)

Epoch 1/50
788/788 [==============================] - 50s 49ms/step - loss: 0.1718 - accuracy: 0.9466 - val_loss: 0.0319 - val_accuracy: 0.9902
Epoch 2/50
788/788 [==============================] - 39s 49ms/step - loss: 0.0556 - accuracy: 0.9825 - val_loss: 0.0422 - val_accuracy: 0.9862
Epoch 3/50
788/788 [==============================] - 47s 59ms/step - loss: 0.0424 - accuracy: 0.9869 - val_loss: 0.0201 - val_accuracy: 0.9939
Epoch 4/50
788/788 [==============================] - 40s 50ms/step - loss: 0.0378 - accuracy: 0.9885 - val_loss: 0.0249 - val_accuracy: 0.9938
Epoch 5/50
788/788 [==============================] - 40s 51ms/step - loss: 0.0332 - accuracy: 0.9894 - val_loss: 0.0245 - val_accuracy: 0.9923
Epoch 6/50
788/788 [==============================] - 39s 49ms/step - loss: 0.0307 - accuracy: 0.9902 - val_loss: 0.0218 - val_accuracy: 0.9931
Epoch 7/50
788/788 [==============================] - 38s 48ms/step - loss: 0.0304 - accuracy: 0.9906 - val_loss: 0.0140 - val_accuracy:

In [ ]:
# Evaluate the model
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

Predict

In [ ]:
result = np.zeros((X_test.shape[0], 10))

result = model.predict(X_test)
result = np.argmax(result, axis=1)
result = pd.Series(result, name="Label")
submission = pd.concat([pd.Series(range(1, 28001), name="ImageId"), result], axis=1)
submission.to_csv("submission.csv", index=False)